In [ ]:
# default_exp load_gti
%load_ext autoreload
%autoreload 2

# load_gti

> Process GTI information

In [ ]:
#export
from astropy.io import fits
import numpy as np
import pandas as pd

from astropy.time import Time, TimeDelta

#mission_start = Time('2001-01-01T00:00:00', scale='utc').mjd
# From a FT2 file header
# MJDREFI =               51910. / Integer part of MJD corresponding to SC clock S
# MJDREFF =  0.00074287037037037 / Fractional part of MJD corresponding to SC cloc
mission_start = 51910.00074287037
day = 24*3600.
first_data=54683

def MJD(met):
    "convert MET to MJD"
    return (mission_start + met/day  )
def UTC(mjd):
    " convert MJD value to ISO date string"
    t=Time(mjd, format='mjd')
    t.format='iso'; t.out_subfmt='date_hm'
    return t.value

def process_gti(self, gti_files):
    """Combine the GTI intervals that fall within the gti range 
    Return a function that tests a list of times 
    """
    if self.verbose>1:
        print(f'Processing {len(gti_files)} GTI files ... ', end='')
    starts=[] 
    stops=[]
    for i, ft1 in enumerate(gti_files):
        with fits.open(ft1) as hdu: 
            gti_data = hdu['GTI'].data
            start = gti_data.START
            if i>0:
                assert start[0]>= stops[-1][-1], f'file {ft1} has start time not following preceding file'
            starts.append(start)
            stops.append( gti_data.STOP)
    start = MJD(np.concatenate(starts))
    stop  = MJD(np.concatenate(stops))

    if self.verbose>1:
        livetime = (stop-start).sum()
        print( f' {len(gti_files)} files, {len(start)} intervals with'\
               f' {int(livetime):,} days live time')

    sel = slice(None)
    if self.mjd_range is not None:
        a, b = self._get_limits(start)
        if a>0 or b<len(start):
            if self.verbose>1:
                print(f'\tcut from {len(start):,} to {a} - {b}, or {b-a:,} entries after MJD range selection')
            sel = slice(a,b)


    class GTI(object):
        """ functor class that tests for being in the GTI range
        """
        def __init__(self, start, stop):
            # prepare single merged array with even, odd entries start and stop
            a,b =start, stop
            self.fraction = np.sum(b-a)/(b[-1]-a[0])
            assert len(a)==len(b)
            self.g = np.array([a,b]).T.flatten()
            assert np.sum(np.diff(self.g)<0)==0, 'Bad GTI ordering'

        def __call__(self, time):
            # use digitize to find if in good/bad interval by odd/even
            x = np.digitize(time, self.g)
            return np.bitwise_and(x,1).astype(bool)

        def __repr__(self):
            return  f'{self.__class__.__name__} MjD range: {self.g[0]:.2f}-{self.g[-1]:.2f}'\
                    f', good fraction {self.fraction:.2f} '

    gti =  GTI(start[sel],stop[sel])
    if self.verbose>1:
        print(f'\t{gti}')
    return gti

In [ ]:
#hide
from light_curves.config import Config, FileConfiguration
config = Config()
gti_files = FileConfiguration().gti
if len(gti_files)>0:
    print(f'Test with GTI files:\n\t {gti_files[0]}\n\t...\n\t{gti_files[-1]}')
    gti_list = process_gti(config, gti_files)
else:
    print('No files, so no test')

Test with GTI files:
	 /home/burnett/work/lat-data/binned/P305_Source_year01_zmax100_4bpd.fits
	...
	/home/burnett/work/lat-data/binned/P305_Source_year11_zmax100_4bpd.fits
Processing 11 GTI files ...  11 files, 63635 intervals with 3,322 days live time
	GTI MjD range: 54682.66-58698.08, good fraction 0.83 


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date